In [1]:
# from FMNM.BS_pricer import BS_pricer
# from FMNM.Parameters import Option_param
# from FMNM.Processes import Diffusion_process
# from FMNM.cython.solvers import PSOR

import numpy as np
import scipy.stats as stats

https://github.com/cantaro86/Financial-Models-Numerical-Methods/tree/master

In [ ]:
N = 4      # number of time steps
r = 0.06   # interest rate
K = 1.1    # strike
T = 3      # Maturity

dt = T / (N - 1)  # time interval
df = np.exp(-r * dt)  # discount factor per time interval

In [ ]:
S = np.array(
    [
        [1.00, 1.09, 1.08, 1.34],
        [1.00, 1.16, 1.26, 1.54],
        [1.00, 1.22, 1.07, 1.03],
        [1.00, 0.93, 0.97, 0.92],
        [1.00, 1.11, 1.56, 1.52],
        [1.00, 0.76, 0.77, 0.90],
        [1.00, 0.92, 0.84, 1.01],
        [1.00, 0.88, 1.22, 1.34],
    ]
)

In [ ]:
https://numpy.org/doc/stable/reference/routines.polynomials.classes.html

In [ ]:
def price_ls(params: dict = None, N=10000, paths=10000, order=2):
    """
        Longstaff-Schwartz Method for pricing American options

        N = number of time steps
        paths = number of generated paths
        order = order of the polynomial for the regression
    """

    # params = {
    S0 = 0
    r = 0
    payoff = None 
    sigma = None

    if payoff != "put":
        raise ValueError("invalid type. Set 'call' or 'put'")

    dt = T / (N - 1)  # time interval
    df = np.exp(-r * dt)  # discount factor per time time interval

    X0 = np.zeros((paths, 1))
    
    increments = stats.norm.rvs(
        loc=(r - sigma**2 / 2) * dt,
        scale=np.sqrt(dt) * sigma,
        size=(paths, N - 1),
    )
    X = np.concatenate((X0, increments), axis=1).cumsum(1)
    S = S0 * np.exp(X)

    H = np.maximum(K - S, 0)  # intrinsic values for put option
    V = np.zeros_like(H)  # value matrix
    V[:, -1] = H[:, -1]

    # Valuation by LS Method
    for t in range(N - 2, 0, -1):
        good_paths = H[:, t] > 0
        rg = np.polyfit(S[good_paths, t], V[good_paths, t + 1] * df, 2)  # polynomial regression
        C = np.polyval(rg, S[good_paths, t])  # evaluation of regression

        exercise = np.zeros(len(good_paths), dtype=bool)
        exercise[good_paths] = H[good_paths, t] > C

        V[exercise, t] = H[exercise, t]
        V[exercise, t+1:] = 0
        discount_path = V[:, t] == 0
        V[discount_path, t] = V[discount_path, t + 1] * df

    V0 = np.mean(V[:, 1]) * df  #
    return V0

N = 4      # number of time steps
r = 0.06   # interest rate
K = 1.1    # strike
T = 3      # Maturity

dt = T / (N - 1)  # time interval
df = np.exp(-r * dt)  # discount factor per time interval